<a href="https://colab.research.google.com/github/smitdesai1010/Flower-Classification-model/blob/master/Flower_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0

In [0]:
from google.colab import files

files.upload()

In [0]:
import json
token = {"username":"smitdesai1010","key":"*****************************"}
with open('/content/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d alxmamaev/flowers-recognition

In [0]:
import zipfile

zf = zipfile.ZipFile('/content/{/content}/datasets/alxmamaev/flowers-recognition/flowers-recognition.zip') 
zf.extractall()

!rm -rf '{' #Deleting the the downloaded Zip files after extracting it 

In [0]:
import tensorflow as tf
import os
import cv2
import datetime
#from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


In [0]:
print(tf.__version__)

In [0]:
def create_dataset(dir):
  path='flowers/'+dir+'/'

  for file in  os.listdir(path):

    img=cv2.imread(path+file , cv2.IMREAD_COLOR)

    try:
     img=cv2.resize(img, (250,250))
    
    except:
     continue 

    images.append(np.array(img))     
    label.append(dir)

In [0]:
images=[]
label=[]

In [0]:
create_dataset('daisy')
create_dataset('dandelion')
create_dataset('rose')
create_dataset('sunflower')
create_dataset('tulip')

In [0]:
print(len(images))
print(len(label))

images=np.asarray(images)

print(images.shape)

In [0]:
label_encoder=LabelEncoder()
integer_encoded=label_encoder.fit_transform(label)

encoded=to_categorical(integer_encoded)

print(encoded.shape)

In [0]:
images=images/255

In [0]:
x_train,x_test,y_train,y_test = train_test_split(images,encoded,test_size=0.25,random_state=40,shuffle=True)

In [0]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

In [0]:
model=tf.keras.models.Sequential([
                                  
tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=x_train[0].shape),
tf.keras.layers.MaxPool2D(pool_size=(2,2)),

tf.keras.layers.Conv2D(64,(4,4),activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPool2D(pool_size=(3,3)),

tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPool2D(pool_size=(3,3)),

tf.keras.layers.Conv2D(32,(4,4),activation='relu'),
tf.keras.layers.MaxPool2D(pool_size=(2,2)),

tf.keras.layers.Flatten(),
tf.keras.layers.Dense(256,activation='relu'),
tf.keras.layers.Dense(64,activation='relu'),
tf.keras.layers.Dense(5,activation='softmax')                                 
                                  
])

In [0]:
model.compile(optimizer='adam',loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [0]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        horizontal_flip=True,  
        vertical_flip=False)  


datagen.fit(x_train)

In [0]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [0]:
model.fit(datagen.flow(x_train,y_train,batch_size=69),epochs=35,validation_data=(x_test,y_test), steps_per_epoch=x_train.shape[0]/69,callbacks=[tensorboard_callback])

In [0]:
%load_ext tensorboard
%tensorboard --logdir logs/fit